In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torchvision
import torchvision.transforms as T
from collections import defaultdict, deque
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import ast
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import torchvision.transforms as transforms

import cv2
import os,sys,matplotlib,re
from PIL import Image
from skimage import exposure
import matplotlib.pyplot as plt
import matplotlib.image as immg
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
import matplotlib

In [ ]:
path= '/content/drive/MyDrive/pred_prof/data/images/64b382d8454dc1c6371508f3b4db3f26dc8ac57a69a2fb935a39a48bfb87551c.png' #путь к фото

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #определяем девайс
device

device(type='cuda')

In [ ]:
#определяем модель и загружаем ее
num_classes = 2 
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/pred_prof/data/epoch3/model_state_epoch_60.pth"))

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

FileNotFoundError: ignored

In [ ]:
#переводим наше изображение в tensor
image = Image.open(path)
transform = transforms.ToTensor()
tensor = transform(image)
print(tensor)

In [ ]:
img = tensor
model.eval()
with torch.no_grad():
    prediction = model([img.to('cuda')])[0]

In [ ]:
def apply_nms(orig_prediction, iou_thresh=0.2):
    
    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

def plot_valid(img,prediction,nms=True,detect_thresh=0.5):
    fig,ax = plt.subplots(figsize=(18,10))
    val_img = img.permute(1,2,0).cpu().numpy()
    ax.imshow(val_img)
    nms_prediction = apply_nms(prediction, iou_thresh=0.2) if nms else prediction
    val_scores = nms_prediction['scores'].cpu().detach().numpy()
    bbox = nms_prediction['boxes'].cpu().detach().numpy()
    cnt = 0
    for i in range(len(bbox)):
        if val_scores[i]>=detect_thresh:
            box = bbox[i]
            x,y,w,h = box[0], box[1], box[2]-box[0], box[3]-box[1]
            rect = matplotlib.patches.Rectangle((x,y),w,h,linewidth=2 ,edgecolor='r',facecolor='none',)
            # ax.text(*box[:2], "wheat {0:.3f}".format(val_scores[i]), verticalalignment='top', color='white', fontsize=12, weight='bold')
            ax.text(*box[:2], "wheat {0:.3f}".format(i), verticalalignment='top', color='white', fontsize=12, weight='bold')
            cnt += 1
            ax.add_patch(rect)
    print('Колосков/боксов:',cnt)
    plt.show()

In [ ]:
plot_valid(img,prediction)